**12 Days of Demos**
# 🎅 From Delta to Delivery-Ready Postgres with Lakebase 🎄

In this notebook, you'll learn how Santa's team syncs Delta tables to Lakebase (Databricks-managed Postgres) so all elf apps can leverage the Lakehouse! Let's unwrap some data magic together! 🎁✨

---

### 🦌 Step 1: Load the North Pole Data

Before we can deliver gifts, we need to load all of the gift requestss! 
These variables set up our Lakehouse catalog, schema, and volume for the holiday data pipeline. 🎅📦

In [0]:
# TODO: Optionally update these values for your environment
TARGET_CATALOG = "main"
TARGET_SCHEMA  = "dbrx_12daysofdemos"
TARGET_VOLUME  = "raw_data_volume"

print(f"✅ Configuration loaded")

In [0]:
%run "../00-init/load-data"

### 🏗️ Step 2: Setup Lakebase for Santa's Workshop

Santa's elves need a magical database engine to power all those wish list lookups! In this section, we'll set up Lakebase (Databricks-managed Postgres) so even the oldest elf apps can join the Lakehouse fun. 🎅🗄️ 

Every workshop needs a strong foundation! Here, we create a Lakebase instance—a magical Postgres engine that powers Santa's data operations. Capacity units are like extra reindeer: more units, more power! 🦌🦌

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.database import DatabaseInstance, DatabaseCatalog, SyncedDatabaseTable, SyncedTableSpec, NewPipelineSpec, SyncedTableSchedulingPolicy, DatabaseInstanceState

# Initialize the Databricks workspace client
w = WorkspaceClient()

# Configure the database instance class
db_instance_config = DatabaseInstance(
    name="lakebase-demo",
    capacity="CU_2",  # Compute capacity: CU_1, CU_2, CU_4, etc.
  )

# Create a new Lakebase database instance
try:
  instance = w.database.create_database_instance(db_instance_config)
  print(f"✓ Instance created: {db_instance_config.name}")
except:
  print(f"Error creating instance or instance already exists")

### 📚 Step 3: Register Santa's Catalog

Now that the Lakebase engine is running, let's organize Santa's data into a shiny new catalog. 
This is where all the elves' tables and schemas will live—neatly wrapped and ready for querying! 🎁📖

In [0]:
# Configure the catalog class
catalog_config = DatabaseCatalog(
        name="lakebase_demo_catalog",                    # Name of the UC catalog to create
        database_instance_name=db_instance_config.name,  # Name of the database instance
        database_name="databricks_postgres",             # Name of the existing Postgres database
    )

# Register an existing database as a UC catalog
if w.database.get_database_instance(db_instance_config.name).state != DatabaseInstanceState.AVAILABLE:
  print(f"Looks like the database instance isn't ready quite yet! Wait 30 seconds and try again!")
else:
  try:
    catalog = w.database.create_database_catalog(catalog_config)
    print(f"✓ Created database catalog: {catalog_config.name}")
  except:
    print(f"Error creating catalog or catalog already exists")

### ✨ Step 4: Create a Synced Table for Santa's Gift Requests

It's time to sync the magic! We'll create a table in Lakebase that stays up-to-date with the elves' latest gift requests. No more manual ETL—just pure holiday automation! 🎄🔄

In [0]:
# Configure the synced table spec class
synced_table_spec = SyncedTableSpec(
            source_table_full_name=TARGET_CATALOG+"."+TARGET_SCHEMA+"."+"gift_requests",
            primary_key_columns=["request_id"],  # Primary key columns
            scheduling_policy=SyncedTableSchedulingPolicy.SNAPSHOT,
            timeseries_key="timestamp",  # For deduplication
            new_pipeline_spec=NewPipelineSpec(
                storage_catalog=TARGET_CATALOG,
                storage_schema=TARGET_SCHEMA
            )
        )

# Configure the synced table class
synced_table_config = SyncedDatabaseTable(
        name=catalog_config.name+".public.gift_requests_synced_table",
        spec=synced_table_spec,
    )

# Create a synced table in a database catalog
try:
  synced_table = w.database.create_synced_database_table(synced_table_config)
  print(f"✓ Created synced table: {synced_table_config.name}")
except:
  print(f"Error creating synced table or synced table already exists")